In [5]:
from pre_process_did import pre_process_did
import pandas as pd, numpy as np
from drdid import drdid, reg_did
from compute_attgt import panel2cs2
import statsmodels.api as sm
import patsy 
fml = patsy.dmatrices

lm = sm.WLS
n_x = np.newaxis
qr_solver = np.linalg.pinv

# from ..aggte.mboot import mboot
from compute_attgt import compute_att_gt

In [7]:
data = pd.read_csv("../../data/sim_data.csv")
data.head()

,Unnamed: 0,G,X,id,period,Y
0,1,4,-0.876233,1,1,3.248701
1,2,4,-0.876233,1,2,2.266837
2,3,4,-0.876233,1,3,3.990885
3,4,4,-0.876233,1,4,4.653489
4,5,4,0.974974,2,1,5.880457


In [8]:
yname = "Y"
tname = "period"
idname = "id"
gname = "G"
data = data
xformla = "Y~X"
dp = pre_process_did(idname=idname, yname=yname, tname=tname, gname=gname, data = data, xformla= xformla)

dp.keys()

dict_keys(['yname', 'tname', 'idname', 'gname', 'xformla', 'data', 'tlist', 'glist', 'n', 'nG', 'nT', 'control_group', 'anticipation', 'weights_name', 'panel', 'true_rep_cross_section', 'clustervar'])

In [9]:
from compute_attgt import  compute_att_gt
op, inf = compute_att_gt(dp)

In [22]:

MP = {
  "DIDparams": dp,
  'n': dp['n'],
  'inffunc': {
    'inffunc': inf
  }
} 
# dp['n']
MP
dp["true_rep_cross_section"]

False

In [33]:
V = np.dot(inf, inf.T) / n
se = np.sqrt(np.diag(V) / n)
eps = np.sqrt(np.finfo(float).eps) * 10
se[se <= eps] = np.nan

### att_gt params
bstrap = True
clustervars = None
cband = True

if (clustervars is not None) & (not bstrap):
  print("clustering the standard errors requires using the bootstrap, resulting standard errors are NOT accounting for clustering")


zero_na_sd_entry = np.where(np.isnan(se))[0]

if bstrap:
  # bout <- mboot(inffunc, DIDparams=dp, pl=pl, cores=cores)
    # bres <- bout$bres
  if len(zero_na_sd_entry) > 0:
    # se[-zero_na_sd_entry] <- bout$se[-zero_na_sd_entry]
    pass
  else:
    pass

se[se <= eps] = np.nan
se

array([0.34778969, 0.4268871 , 0.41376458, 0.37886667, 0.32701394,
       0.40092314, 0.46239724, 0.46317022, 0.50349182])

In [544]:
####### compute att_gt params
est_method = "dr"
base_period = "varying"


yname = dp['yname']
tname = dp['tname']
idname = dp['idname']
xformla = dp['xformla']
data = dp['data']
weights_name = dp['weights_name']
# base_period = dp['base_period']
panel = dp['panel']
true_rep_cross_section = dp['true_rep_cross_section']
control_group = dp['control_group']
anticipation = dp['anticipation']
gname = dp['gname']
n = dp['n']
nT = dp['nT']
nG = dp['nG']
tlist = dp['tlist']
glist = dp['glist']

In [545]:
tlist_len, tfac = len(tlist), 0
if base_period != 'universal':
  tlist_len = tlist_len - 1
  tfac = 1

inf_func = []

att_est, group, year, post_array = [], [], [], []

def add_att_data(att = 0, pst = 0, inf_f = []):
  inf_func.append(inf_f)
  att_est.append(att)
  group.append(g)
  year.append(tn)
  post_array.append(pst)

never_treated = control_group == 'nevertreated'
if never_treated:
  data = data.assign(C = 1 * (data[gname] == 0))
data = data.assign(y_main = data[yname])

In [546]:
g_i = 2
t_i = 2
g, t = glist[g_i], tlist[t_i]
g, t

(4, 3)

In [547]:
len(glist), len(tlist)

(3, 4)

### For loop G

In [548]:
G_main = (data[gname] == g)
# for _, g, in enumerate(glist)
data = data.assign(G_m = 1 * G_main)

#### for loop T 

In [549]:

# for _, t, in enumerate(glist)
pret = t_i
tn = tlist[t_i + tfac]
if base_period == 'universal' or g < tn:
  print("here")
  try:
    pret = np.where(tlist + anticipation < g)[0][-1]
  except:
    raise f"There are no pre-treatment periods for the group first treated at {g}\nUnits from this group are dropped"
    # break


if base_period == 'universal':
  if pret == tn:
    add_att_data()

if not never_treated:
  n1 = data[gname] == 0
  n2 = (data[gname] > (tlist[np.max([t, pret]) + tfac]) + anticipation)
  n3 = np.where(data[gname] != glist[g], True, False)
  row_eval = n1 | n2 & n3
  data = data.assign(C = 1 * row_eval)

post_treat = 1 * (g <= tn)
disdat = data[(data[tname] == tn) | (data[tname] == tlist[pret])]
pret, tn, post_treat, disdat.shape

(2, 4, 1, (160, 11))

# panel: True

In [550]:
disdat = panel2cs2(data=disdat, yname=yname, idname=idname, tname=tname)

In [551]:
disdat = disdat.dropna()
n = len(disdat)
dis_idx = np.array(disdat.G_m == 1) | np.array(disdat.C == 1)
disdat = disdat.loc[dis_idx, :]
n1 = len(disdat)
G = disdat.G_m
C = disdat.C
w = disdat.w

In [552]:
ypre = disdat.y0 if tn > pret else disdat.y1
ypost = disdat.y0 if tn < pret else disdat.y1
_, covariates = fml(xformla, data = disdat, return_type = 'dataframe')

G, C, w, ypre = map(np.array, [G, C, w, ypre])
ypost, covariates = map(np.array, [ypost, covariates])

In [553]:
#todo: https://github.com/bcallaway11/did/blob/master/R/compute.att_gt.R
# 192 - 224

In [554]:
est_method = 'reg'

if callable(est_method):
  est_att_f = est_method
elif est_method == "reg":
  est_att_f = reg_did.reg_did_panel
elif est_method == "dr":
  est_att_f = drdid.drdid_panel

att_gt, att_inf_func = est_att_f(ypost, ypre, G, i_weights=w, covariates=covariates)

inf_zeros = np.zeros(n)
att_inf = n / n1 * att_inf_func
inf_zeros[dis_idx] = att_inf

add_att_data(att_gt, inf_f=inf_zeros)

## reg did panel

In [555]:
y1, y0 = ypost, ypre
D = G
i_weights = w

In [556]:
delta_y = y1 - y0
rows = D == 0
int_cov = covariates
reg_coef = lm(delta_y[rows], int_cov[rows], weights=i_weights[rows]).fit().params
out_delta = np.dot(reg_coef, int_cov.T)
w_treat = i_weights * D
w_cont = w_treat.copy()

reg_att_treat = w_treat * delta_y
reg_att_cont = w_cont * out_delta

eta_treat = np.mean(reg_att_treat) / np.mean(w_treat)
eta_cont = np.mean(reg_att_cont) / np.mean(w_cont)

reg_att = eta_treat - eta_cont
reg_att

1.4356558918370697

In [557]:
def asy_lin_wols(d, post, y, out_y, int_cov = int_cov):
  weigths_ols = i_weights * d * post
  # weigths_ols_pre
  wols_x = weigths_ols[:, n_x] * int_cov
  wols_ex = (weigths_ols * (y - out_y))[:, n_x] * int_cov
  cr = np.dot(wols_x.T, int_cov) / n
  xpx_inv = qr_solver(cr)
  asy_lin_rep_ols = np.dot(wols_ex, xpx_inv)
  return asy_lin_rep_ols

In [558]:
d = 1 - D
post = 1

asy_lin_rep_ols = asy_lin_wols(d, post, delta_y, out_delta)
inf_treat = (reg_att_treat - w_treat * eta_treat) / np.mean(w_treat)
inf_cont_1 = (reg_att_cont - w_cont * eta_cont)

M1 = np.mean(w_cont[:, n_x] * int_cov, axis = 0)
inf_cont_2 = np.dot(asy_lin_rep_ols, M1)
inf_control = (inf_cont_1 + inf_cont_2) / np.mean(w_cont)

reg_att_inf_func = inf_treat - inf_control
reg_att_inf_func

array([-1.09771797,  1.27100744,  1.21225699,  1.40133318, -1.71640218,
        0.29437419,  2.39402706,  2.72359389, -3.87620441, -1.61023425,
       -3.59369111,  0.49710258,  2.63541817,  0.05664299,  3.70033684,
        0.5143974 , -5.24655528, -3.19039958,  1.87746232,  2.1704325 ,
        6.75331284,  3.68043035, -2.25087799, -4.11849979, -4.48154615,
        5.12889684, -3.24129817, -7.32056705, -7.62327415,  0.35877505,
       -0.99692526,  5.30770195,  4.55055024, -0.10975528,  0.58788976,
        5.25377453,  8.3399879 , -8.87739825, -2.09778414,  9.91329921,
       -0.55178127, -4.24149996,  5.6733284 , -5.03025925, -5.02366108])

# panel: False

In [564]:
right_ids = np.array(disdat.query('(G_m == 1) or (C == 1)').rowid)
dis_idx = (data['rowid'].isin(right_ids)) &\
   ((data[tname] == tlist[t_i + tfac]) |\
    (data[tname] == tlist[pret]))

disdat = data.loc[dis_idx]

G = disdat.G_m
C = disdat.C
Y = disdat[yname]
post = 1 * (disdat[tname] == tlist[t_i + tfac])
w = disdat.w

G, C, Y, post, w = map(np.array, [G, C, Y, post, w])

n1 = sum(G + C)
# right_ids, n1, w, sum(dis_idx), post

In [565]:
skip_this_att_gt = False
if np.sum(G * post) == 0:
  print(f"No units in group {g} in time period {tn}")
  skip_this_att_gt = True 

if np.sum(G * (1 - post)) == 0:
  print(f"No units in group {g} in time period {t}")
  skip_this_att_gt = True 

if np.sum(C * post) == 0:
  print(f"No available control units for group {g} in time period {tn}")
  skip_this_att_gt = True 

if np.sum(C * (1 - post)) == 0:
  print(f"No available control units for group {g} in time period {t}")
  skip_this_att_gt = True 

if skip_this_att_gt:
  add_att_data()


In [566]:

_, covariates = fml(xformla, data = disdat, return_type = 'dataframe')
covariates = np.array(covariates)

### Drdid rc

<!-- pip install git+https://github.com/d2cml-ai/drdid -->

In [567]:
est_method = 'dr'

if callable(est_method):
  est_att_f = est_method
elif est_method == "reg":
  est_att_f = reg_did.reg_did_rc
elif est_method == "dr":
  est_att_f = drdid.drdid_rc
att_gt, att_inf_func = est_att_f(y=Y, post=post, D = G, i_weights=w, covariates=covariates)
att_gt


1.439661425578744

In [568]:
att_inf_func

array([ -3.63259846,   1.1664611 ,   1.29433975,   1.44833396,
        -5.67936218,   7.83384784,  -4.46782381,   6.82872427,
        -2.44575645,  -0.64437655,  -1.86260136,   2.54960207,
        -3.186556  ,   7.84683507,  -5.47304655,  10.91339322,
        -2.4492188 ,  -5.39258055,   3.32673139,  -6.50287041,
        -9.16003951,   1.94394126,  -2.63833872,   4.08319751,
         4.60359905,   0.60470802,   8.90182018,  -8.3404923 ,
        24.50314832, -17.03344469,   4.62041087,  -3.49382355,
       -14.07376675,   3.49939693,   1.47956481,  -8.00205126,
        -8.55790577,  12.68662483,  10.74334689,  -6.72194936,
         5.76600388,   7.74022807,   1.65591968,   5.79757346,
         5.55101947, -10.36472024,  -4.44198406,  -4.12655721,
        -4.37168124,  -4.31477683,   1.14128205,   4.5711576 ,
       -10.21700327,   6.17398764,  -4.06781146,  -4.33627892,
         7.8321716 , -15.99859838,  -1.53553919,   2.04897166,
       -14.44322043,  13.74179397,   6.02260821,  -0.55

In [580]:
for gi, g in enumerate(glist):
  for t_i, t in enumerate(tlist):
    print(f"{g} - {t}")

2 - 1
2 - 2
2 - 3
2 - 4
3 - 1
3 - 2
3 - 3
3 - 4
4 - 1
4 - 2
4 - 3
4 - 4


In [583]:
tlist

array([1, 2, 3, 4], dtype=int64)

In [579]:
inf_func_df = pd.DataFrame(
  {
    "inf_func": att_inf_func,
    "right_ids": right_ids
  }
)
len(att_inf_func), len(right_ids)
inf_zeros = np.zeros(n)
aggte_infffuc = inf_func_df.groupby('right_ids').inf_func.sum()
dis_idx1 = np.isin(data['rowid'].unique(), aggte_infffuc.index)
inf_zeros[dis_idx1] = np.array(aggte_infffuc)

# add_att_data(att_gt, pst = post_treat, inf_f=inf_zeros)
# ## End panel false

In [578]:
# data Struct
output = {
  "att" : att_est,
  'group': group ,
  'year': year,
  'post ': post_array
}
output, np.array(inf_zeros)


({'att': [1.4356558918370697], 'group': [4], 'year': [4], 'post ': [0]},
 array([ -2.46613736,   2.74267371,   2.15448566,   0.        ,
          0.        ,   0.        ,   2.36090046,  -3.090133  ,
          0.        ,   0.68700071,   4.66027907,   0.        ,
          0.        ,   0.        ,   5.44034667,   0.        ,
          0.        ,  -7.84179935,  -3.17613903,   0.        ,
          0.        ,   0.        ,   0.        ,  -7.21609825,
          0.        ,   0.        ,   1.44485879,   5.20830706,
          0.56132788,   7.46970363,   0.        ,   0.        ,
          1.12658733, -10.57436982,   0.        ,  -6.52248645,
          4.12871906,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   4.02139753,
         13.50623195,   0.        ,   0.        ,   7.45349314,
          0.        ,   0.        ,  -4.81370078,   0.        ,
          0.        ,   0.        ,   0.        ,  -8.56854127,
          0.        ,   0.     

In [89]:
int_cov = np.ones(n)
if covariates is not None:
  if np.all(covariates[:, 0] == int_cov):
    int_cov = covariates
  else:
    int_cov = np.concatenate((np.ones((n, 1)), covariates), axis=1)

C:\Users\Jhon\AppData\Local\Temp\ipykernel_7100\467515111.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(covariates[:, 0] == int_cov):


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 80 and the array at index 1 has size 66

In [ ]:
pscore_tr = glm(D, int_cov, family=binomial, freq_weights=i_weights)\
  .fit()
ps_fit = pscore_tr.fittedvalues
ps_fit = np.minimum(ps_fit, 1 - 1e-16)

def reg_out_y(d, p, y = y, int_cov = int_cov, wg = i_weights):
  rows_ = (D == d) & (post == p)
  reg_cont = lm(y[rows_], int_cov[rows_], weights=wg[rows_])\
    .fit().params
  out_y = np.dot(reg_cont, int_cov.T)
  return out_y


out_y_cont_pre = reg_out_y(d = 0, p = 0)
out_y_cont_post = reg_out_y(d = 0, p = 1)
out_y_cont = post * out_y_cont_post + (1 - post) * out_y_cont_pre 


out_y_treat_pre = reg_out_y(d = 1, p = 0)
out_y_treat_post = reg_out_y(d = 1, p = 1)

w_treat_pre = i_weights * D * (1 - post)
w_treat_post = i_weights * D * post
rest_cont = i_weights * ps_fit * (1 - D) 
w_cont_pre = rest_cont * (1 - post) / (1 - ps_fit)
w_cont_post = rest_cont * post / (1 - ps_fit)


w_d = i_weights * D
w_dt1 = w_d * post
w_dt0 = w_d * (1 - post)

def eta_treat(a, b = out_y_cont, y = y):
  return a * (y - b) / np.mean(a)

eta_treat_pre = eta_treat(w_treat_pre)
eta_treat_post = eta_treat(w_treat_post)
eta_cont_pre = eta_treat(w_cont_pre)
eta_cont_post = eta_treat(w_cont_post)

eta_d_post = eta_treat(w_d, out_y_cont_post, out_y_treat_post)
eta_d_pre = eta_treat(w_d, out_y_cont_pre, out_y_treat_pre)
eta_dt1_post = eta_treat(w_dt1, out_y_cont_post, out_y_treat_post)
eta_dt0_pre = eta_treat(w_dt0, out_y_cont_pre, out_y_treat_pre)

att_treat_pre = np.mean(eta_treat_pre)
att_treat_post = np.mean(eta_treat_post)
att_cont_pre = np.mean(eta_cont_pre)
att_cont_post = np.mean(eta_cont_post)

att_d_post = np.mean(eta_d_post)
att_dt1_post = np.mean(eta_dt1_post)
att_d_pre = np.mean(eta_d_pre)
att_dt0_pre = np.mean(eta_dt0_pre)


dr_att = (att_treat_post - att_treat_pre) - \
  (att_cont_post - att_cont_pre) -\
  (att_d_post - att_dt1_post) -\
  (att_d_pre - att_dt0_pre)

In [ ]:
def asy_lin_wols(d, post, out_y, int_cov = int_cov):
  weigths_ols = i_weights * d * post
  # weigths_ols_pre
  wols_x = weigths_ols[:, n_x] * int_cov
  wols_ex = (weigths_ols * (y - out_y))[:, n_x] * int_cov
  cr = np.dot(wols_x.T, int_cov) / n
  xpx_inv = qr_solver(cr)
  asy_lin_rep_ols = np.dot(wols_ex, xpx_inv)
  return asy_lin_rep_ols

asy_lin_rep_ols_pre = asy_lin_wols((1 - D), (1 - post), out_y_cont_pre)
asy_lin_rep_ols_post = asy_lin_wols((1 - D), post, out_y_cont_post)

asy_lin_rep_ols_pre_treat = asy_lin_wols(
  D, (1 - post), out_y_treat_pre
)
asy_lin_rep_ols_post_treat = asy_lin_wols(D, post, out_y_treat_post)


In [ ]:
score_ps = (i_weights * (D - ps_fit))[:, n_x] * int_cov
hessian_ps = pscore_tr.cov_params() * n 
asy_lin_rep_ps = np.dot(score_ps, hessian_ps)

inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre\
  / mean(w_treat_pre)
inf_treat_post = eta_treat_post - w_treat_post * att_treat_post\
  / mean(w_treat_post)

M1_post = np.mean((w_treat_post * post)[:, n_x] * int_cov, axis=0) / np.mean(w_treat_post)
M1_pre = np.mean((w_treat_pre * (1 - post))[:, n_x] * int_cov, axis=0) / np.mean(w_treat_pre)


In [ ]:
inf_treat_or_post = np.dot(asy_lin_rep_ols_post, M1_post)
inf_treat_or_pre = np.dot(asy_lin_rep_ols_pre, M1_pre)
inf_treat_or = inf_treat_or_post - inf_treat_or_pre

inf_treat = inf_treat_post - inf_treat_pre + inf_treat_or

inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)


In [ ]:

M2_pre = np.mean(
  ((w_cont_pre * (y - out_y_cont - att_cont_pre))[:, n_x] * int_cov), 
  axis=0
) / np.mean(w_cont_pre)
M2_post = np.mean(
  ((w_cont_post * (y - out_y_cont - att_cont_post))[:, n_x] * int_cov),
  axis=0
) / np.mean(w_cont_post)

inf_cont_ps = np.dot(asy_lin_rep_ps, M2_post - M2_pre)

In [ ]:
M3_post = np.mean(
  (w_cont_post * post)[:, n_x] * int_cov, axis=0
) / np.mean(w_cont_post)
M3_pre = np.mean(
  (w_cont_pre * (1 - post))[:, n_x] * int_cov, axis=0
) / np.mean(w_cont_pre)

inf_cont_or_post = np.dot(asy_lin_rep_ols_post, M3_post)
inf_cont_or_pre = np.dot(asy_lin_rep_ols_pre, M3_pre)
inf_cont_or = inf_cont_or_post - inf_cont_or_pre

inf_cont = inf_cont_post - inf_cont_pre + inf_cont_ps + inf_cont_or

dr_att_inf_func1 = inf_treat - inf_cont

def inf_eff_f(a, b, c):
  return a - b * c / np.mean(b)

inf_eff1 = inf_eff_f(eta_d_post, w_d, att_d_post)
inf_eff2 = inf_eff_f(eta_dt1_post, w_dt1, att_dt1_post)
inf_eff3 = inf_eff_f(eta_d_pre, w_d, att_d_pre)
inf_eff4 = inf_eff_f(eta_dt0_pre, w_dt0, att_dt0_pre)
inf_eff = inf_eff1 - inf_eff2 - (inf_eff3 - inf_eff4)

def mom_f(a, b, int_cov = int_cov):
  left = a / np.mean(a) - b / np.mean(b)
  return np.mean(left[:, n_x] * int_cov, axis=0) 

mom_post = mom_f(w_d, w_dt1)
mom_pre = mom_f(w_d, w_dt0)

inf_or_post = np.dot(
  asy_lin_rep_ols_post_treat - asy_lin_rep_ols_post, 
  mom_post
)
inf_or_pre = np.dot(
  asy_lin_rep_ols_pre_treat - asy_lin_rep_ols_pre, 
  mom_pre
)
inf_or = inf_or_post - inf_or_pre
dr_att_inf_func = dr_att_inf_func1 + inf_eff + inf_or
dr_att

0.8363102379527425